<a href="https://colab.research.google.com/github/jecampagne/JaxTutos/blob/main/JAX_JIT_in_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import scipy as sc


import jax
import jax.numpy as jnp
import jax.scipy as jsc   

from jax import grad, jit, vmap
from jax import jacfwd, jacrev, hessian
#from jax.ops import index, index_update
jax.config.update("jax_enable_x64", True)

from functools import partial
from typing import Union, Dict, Callable, Optional, Tuple, Any

In [4]:
jax.__version__

'0.4.4'

In [ ]:
jax._src.

In [5]:
def u_print(idx:int, *args)->int:
    print(f"{idx}):",*args)
    idx +=1
    return idx


# Topic: JIT and Class methods

This notebook is a bit technical but deals with a real problem when one comes from the C++ world.

it's an element of a JAX thread that I initiated and that becomes a part of the doc [see FAQ](https://jax.readthedocs.io/en/latest/faq.html#strategy-1-jit-compiled-helper-function)

In [9]:
class A():
    def __init__(self, a: float):
        print("New A")
        self.a = a                   # a variable that will be "static" once the obj is created
        self.b = None                # a variable ("dynamic")

    def f(self, var: float) -> None:
        self.b = self.a * var

###### 

idp=0

objA = A(2.0)
idp = u_print(idp,objA.a, objA.b)
objA.f(10.)
idp = u_print(idp,objA.a, objA.b)
objA.f(11.)
idp = u_print(idp,objA.a, objA.b)


objA = A(3.0)
idp = u_print(idp,objA.a, objA.b)
objA.f(20.)
idp = u_print(idp,objA.a, objA.b)

New A
0): 2.0 None
1): 2.0 20.0
2): 2.0 22.0
New A
3): 3.0 None
4): 3.0 60.0


# And now use jit on f

In [10]:
class A():
    def __init__(self, a: float):
        self.a = a                   # une variable qui ne changera pas ("statique")
        self.b = None                # une variable qui sera déterminée utltérieuremnt ("dynamique")
    @jit
    def f(self, var: float) -> None:
        self.b = self.a * var


idp=0

objA = A(2.0)
idp = u_print(idp,objA.a, objA.b)
objA.f(10.)


0): 2.0 None


TypeError: ignored

The problem is that the first argument of the function is `self`, which has the type `A`and JAX does not know what it is.

There are different basic strategies we can use in this case, and we will discuss them below.

# 1st strategy: external function ('helper')

## small aside...
"in Python, type annotations are purely decorative and don't affect runtime values in normal code"

In [13]:
print(1.0*jnp.array([10.]))
def test(v: jnp.array)->jnp.array:
    return 1.0*v
print(test(jnp.array([10.])))
print(test(10.))

[10.]
[10.]
10.0


In [15]:
@jit
def f(v):
    return 1.0 * v
print(repr(f(10.0)), type(f(10.0)))  # implicit conversion d'un float en JAX device array (a-la-jnp.array)


Array(10., dtype=float64, weak_type=True) <class 'jaxlib.xla_extension.Array'>


In [20]:
class A():
    def __init__(self, a):
        print("new A")
        self.a = a   
        self.b = None 
   
    def f(self, var):
        self.b = _f(self.a,var)

@jit
def _f(a:float, var):
    print("compile...")
    res = a* var
    return res



idp=0

print("New Obj A (1)")
objA = A(2.0)
idp = u_print(idp, objA.a, objA.b)
objA.f(10.)
idp = u_print(idp, objA.a, objA.b)
objA.f(11.)
idp = u_print(idp, objA.a, objA.b)


print("New Obj A (2)")
objA = A(3.0)
idp = u_print(idp, objA.a, objA.b)
objA.f(20.)
idp = u_print(idp, objA.a, objA.b)
objA.f(21.)
idp = u_print(idp, objA.a, objA.b)


print("New Obj A (3)")
objA = A(4.0)
idp = u_print(idp, objA.a, objA.b)
objA.f(20.)
idp = u_print(idp, objA.a, objA.b)

print("f is called with an array not a float => compile")
objA.f(jnp.array([20.]))
idp = u_print(idp, objA.a, objA.b)

objA.f(jnp.array([21.]))
idp = u_print(idp, objA.a, objA.b)

objA.f(21.)
idp = u_print(idp, objA.a, objA.b)

print("change internal objA.a")

objA.a = 400.  # change
objA.f(21.)
idp = u_print(idp, objA.a, objA.b)


New Obj A (1)
new A
0): 2.0 None
compile...
1): 2.0 20.0
2): 2.0 22.0
New Obj A (2)
new A
3): 3.0 None
4): 3.0 60.0
5): 3.0 63.0
New Obj A (3)
new A
6): 4.0 None
7): 4.0 80.0
f is called with an array not a float => compile
compile...
8): 4.0 [80.]
9): 4.0 [84.]
10): 4.0 84.0
change internal objA.a
11): 400.0 8400.0


## Review of the helper method

- This is a simple method
implementation, and we don't have to instruct JAX how to use the A class.

- Now it becomes a matter of taste to code a helper per function to use jit. But we can do file encapsulation so that at least the code of "A" is in the code of the definition of "A".

- Pb: what if the function f needs for example another function (either member of A, or external to A)?

# 2nd strategy: self as static


it is a classic procedure that is often proposed

In [24]:
class A():
    def __init__(self, a: float):
        print("new A")
        self.a = a
        self.b = None
    
    @partial(jit, static_argnums=(0,))   # "self" is static    
    def f(self, var: float) -> None:
        print("compile...")
        self.b = self.a * var  #<- this is the culpit of the later crash on "g"

    def g(self):
        print("g...:",self.b)
        return self.b*self.b

idp=0

objA = A(2.0)
idp = u_print(idp, objA.a, objA.b)
objA.f(10.)
idp = u_print(idp, objA.a, objA.b)
res = objA.g()   # will case a crash but this is not the culpit 
print(res)


new A
0): 2.0 None
compile...
1): 2.0 1.0
g...: 1.0
1.0


The code crash and seems to point the `g` function BUT in fact the problem comes from the jitted `f` function with self as static: it uses Traced<ShapedArray> to analyze the code, and propagate...

Do not use `self.var =` in a jitted function with self as static.

In [26]:
class A():
    def __init__(self, a):
        print("Nouveau A")
        self.a = a               
        self.b = None            

    def set_a(self,x):
        self.a = x 
        print("new a:",self.a)
        
    def set_b(self,x):                  # (*) call from extern
        self.b = x
    
    @partial(jit, static_argnums=(0,))   #  "self" is static
    def f(self, var):
        print("compile...")
        return self.a * var             # see (*)

    def g(self):
        print("g...:",self.b)
        return self.b*self.b


idp=0

objA = A(2.0)
idp = u_print(idp, objA.a, objA.b)
b  = objA.f(10.)
objA.set_b(b)      #<----------------------
idp = u_print(idp, objA.a, objA.b)
res = objA.g() 
print(res)


Nouveau A
0): 2.0 None
compile...
1): 2.0 20.0
g...: 20.0
400.0


## We have to be careful:
- the "static" character of "a" is a by-product of the analysis of the code by JIT, but this is done silently and without warning and therefore, this can mislead the user

What is behind? The problem is that ``tatic_argnums` relies on the hash method of the object to determine if it has changed between two calls, and the
method `__hash__` for a user-defined class does not take into account the class attributes.

This means that on the second function call, JAX has no way of knowing that the class attributes have changed and uses the static value cached during the previous compilation.

For this reason, if you mark personal arguments as static, it is important that you define an appropriate `__hash__` method for your class. For example:

In [29]:
class A():
    def __init__(self, a):
        print("new A")
        self.a = a
        self.b = None
        
    def set_a(self,x):
        self.a = x 
        print("new a:",self.a)

        
    def set_b(self,x):
        self.b = x
    
    @partial(jit, static_argnums=0)   #"self" is static
    def f(self, var):
        print("compile...")
        return self.a * var
    
    # specific hash metho
    def __hash__(self):
        return hash((self.a,self.b))

    def __eq__(self, other):   # you need it also
        return (isinstance(other, A) and
            (self.a, self.b) == (other.a, other.b))
    


But this is not the end....

In [30]:
idp=0

objA = A(2.0)
idp = u_print(idp, objA.a, objA.b)

b = objA.f(10.)         #  quel est le type de 'b' ??? (*)
print("b:", b, type(b))
objA.set_b(b)
idp = u_print(idp, objA.a, objA.b)

objA.f(11.)

new A
0): 2.0 None
compile...
b: 20.0 <class 'jaxlib.xla_extension.Array'>
1): 2.0 20.0


ValueError: ignored

the problem comes from the fact that after compilation `b` becomes a `DeviceArray` which is unhastable so it's stuck because then self can't be hashtable too !!!

In [31]:
class A():
    def __init__(self, a):
        print("Nouveau A")
        self.a = a
        self.b = None

    def set_a(self,x):
        self.a = x 
        print("new a:",self.a)

    def set_b(self,x):
        self.b = x
    
    @partial(jit, static_argnums=0)
    def f(self, var):
        print("compile...")
        return self.a * var
    
    def __hash__(self):
        return hash((self.a))                 # do not put "b"

    def __eq__(self, other):
        return (isinstance(other, A) and
            (self.a) == (other.a))             # do not put "b"


In [32]:
idp=0

objA = A(2.0)
idp = u_print(idp, objA.a, objA.b)

b = objA.f(10.)         #  quel est le type de 'b' ??? (*)
print("b:", b, type(b))
objA.set_b(b)
idp = u_print(idp, objA.a, objA.b)

objA.set_b(objA.f(11.))
idp = u_print(idp, objA.a, objA.b)


objA = A(3.0)
idp = u_print(idp, objA.a, objA.b)
objA.set_b(objA.f(20.))
idp = u_print(idp, objA.a, objA.b)

Nouveau A
0): 2.0 None
compile...
b: 20.0 <class 'jaxlib.xla_extension.Array'>
1): 2.0 20.0
2): 2.0 22.0
Nouveau A
3): 3.0 None
compile...
4): 3.0 60.0


In [34]:
objA.set_a(40.)
objA.set_b(objA.f(11.))
idp = u_print(idp, objA.a, objA.b)   # 40.*11. = 440


new a: 40.0
compile...
6): 40.0 440.0


## Review self as static:

- We have to carefully define `__hass__`  and `__eq__` (see Python [doc](https://docs.python.org/3/reference/datamodel.html#object.__hash__)) but DeviceArray are non-hashtable

- `a` can be redefined voluntarily (with/without setter), the result of `b` is well updated

- So can be done but with care and there is a simpler solution


# 3d strategy: PyTree

In [70]:
class A():
    def __init__(self, a, b=None):   # new signature
        #print("Nouveau A")
        self.a = a
        self.b = b

        
    def set_a(self,x):
        self.a = x 
        print("new a:",self.a)

    def set_b(self, b):
        self.b = b
        
    @jit                              # <------ self, no more static    !                
    def f(self, var):
        print("compile...")
        return self.a * var

    def g(self):
      return self.b * self.b

    #### PyTree methods....
    def tree_flatten(self):
        children = (self.b,)         # dynamic values
        aux_data = {'a': self.a}     # static values
        return (children, aux_data)

    @classmethod                     
    def tree_unflatten(cls, aux_data, children):
        b = children
        a = aux_data['a']
        return cls(a=a, b=b ) # should be the signature of __init__

# register explicit Or decorator
from jax import tree_util
tree_util.register_pytree_node(A,
                               A.tree_flatten,
                               A.tree_unflatten) 

In [71]:
idp=0

objA = A(2.0)
idp = u_print(idp, objA.a, objA.b)

b = objA.f(10.)         #  quel est le type de 'b' ??? (*)
print("b:", b, type(b))
objA.set_b(b)
idp = u_print(idp, objA.a, objA.b)

objA.set_b(objA.f(11.))
idp = u_print(idp, objA.a, objA.b)


objA = A(3.0) 
idp = u_print(idp, objA.a, objA.b)
objA.set_b(objA.f(20.))
idp = u_print(idp, objA.a, objA.b)
objA.set_b(objA.f(30.))
idp = u_print(idp, objA.a, objA.b)


objA.set_a(30.) 
objA.set_b(objA.f(30.))
idp = u_print(idp, objA.a, objA.b)



objA = A(3.0) 
idp = u_print(idp, objA.a, objA.b)

objA.set_b(objA.f(20.))
idp = u_print(idp, objA.a, objA.b)

objA.set_b(objA.f(30.))
idp = u_print(idp, objA.a, objA.b)

print(">>>> try an array for var")

objA.set_b(objA.f(jnp.array([30.])))
idp = u_print(idp, objA.a, objA.b)

objA.set_b(objA.f(jnp.array([40.])))
idp = u_print(idp, objA.a, objA.b)

print("g: ", objA.g())

0): 2.0 None
compile...
b: 20.0 <class 'jaxlib.xla_extension.Array'>
1): 2.0 20.0
compile...
2): 2.0 22.0
3): 3.0 None
compile...
4): 3.0 60.0
compile...
5): 3.0 90.0
new a: 30.0
compile...
6): 30.0 900.0
7): 3.0 None
8): 3.0 60.0
9): 3.0 90.0
>>>> try an array for var
compile...
10): 3.0 [90.]
compile...
11): 3.0 [120.]
g:  [14400.]


## A possible variation

In [69]:
class A():
    def __init__(self, a, b=None):
        self.a = a
        self.b = b

    @jit                                       
    def f(self, var):
        print("compile...")
        new_b = self.a * var
        return A(self.a, new_b)       # <--------- new objet.


# registration with  lambda functions    
tree_util.register_pytree_node(A,
                               lambda x: ((x.a,x.b), None),
                               lambda _, x: A(a=x[0],b=x[1]) 
                              )

In [42]:
idp=0

objA = A(2.0)
idp = u_print(idp, objA.a, objA.b)    #0

objA = objA.f(10.)
idp = u_print(idp,objA.a, objA.b)     #1

objA = objA.f(11.)
idp = u_print(idp, objA.a, objA.b)    #2

####

objA = A(3.0) 
idp = u_print(idp, objA.a, objA.b)     #3
objA = objA.f(20.)
idp = u_print(idp, objA.a, objA.b)     #4
objA= objA.f(30.)
idp = u_print(idp, objA.a, objA.b)     #5


objA.a = 30.  # change 
objA= objA.f(30.)
idp = u_print(idp, objA.a, objA.b)     #6

####

objA = A(3.0) 
idp = u_print(idp, objA.a, objA.b)     #7

objA= objA.f(20.)
idp = u_print(idp, objA.a, objA.b)     #8

objA= objA.f(30.)
idp = u_print(idp, objA.a, objA.b)     #9

print(">>>> try an array for var")

objA= objA.f(jnp.array([30.]))
idp = u_print(idp, objA.a, objA.b)     #10

objA= objA.f(jnp.array([40.]))
idp = u_print(idp, objA.a, objA.b)     #11

objA= objA.f(jnp.array([50.]))         
idp = u_print(idp, objA.a, objA.b)     #12


objA.a = 40. ### new change
objA= objA.f(jnp.array([50.]))
idp = u_print(idp, objA.a, objA.b)     #13


0): 2.0 None
1): 2.0 20.0
2): 2.0 22.0
3): 3.0 None
4): 3.0 60.0
5): 3.0 90.0
6): 30.0 900.0
7): 3.0 None
8): 3.0 60.0
9): 3.0 90.0
>>>> try an array for var
10): 3.0 [90.]
11): 3.0 [120.]
12): 3.0 [150.]
13): 40.0 [2000.]


## Review PyTree

- one must somehow set up `tree_flatten` and `tree_unflatten` with care
- the change of ``a` is well transmitted
- choice of implementation with new object each time or setter of the internal variable to change

In [114]:
from jax.tree_util import register_pytree_node_class

@register_pytree_node_class
class B():
    def __init__(self, p, v):
        self.p = p
        self.v = v
        print("init:",p,v)

    @jit
    def step(self, dt):
        print("compile...")
        a = -9.8
        new_v = self.v + a * dt
        new_p = self.p + new_v * dt
        return B(new_p, new_v)
    
    #### PyTree methods....
    def tree_flatten(self):
        print("tree_flatten")
        children = (self.p,self.v)         # dynamic values
        aux_data = {}     # static values
        return (children, aux_data)

    @classmethod                     
    def tree_unflatten(cls, aux_data, children):
        print("tree_unflatten") 
        b = children
        return cls(p=b[0], v=b[1] ) # should be the signature of __init__



In [122]:
b = B(jnp.array([0., 0.5]), jnp.array([1., 1.5]))

init: [0.  0.5] [1.  1.5]


In [123]:
for i in range(5):
    b = b.step(0.01)


tree_flatten
tree_unflatten
init: [0.00902 0.51402] [0.902 1.402]
tree_flatten
tree_unflatten
init: [0.01706 0.52706] [0.804 1.304]
tree_flatten
tree_unflatten
init: [0.02412 0.53912] [0.706 1.206]
tree_flatten
tree_unflatten
init: [0.0302 0.5502] [0.608 1.108]
tree_flatten
tree_unflatten
init: [0.0353 0.5603] [0.51 1.01]


In [124]:
! pip install -q equinox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 KB 3.2 MB/s eta 0:00:00


In [125]:
import equinox as eqx

In [138]:
class D(eqx.Module):
  p: float
  v: float

  def __init__(self, p,v):  # the init
    self.p = p
    self.v = v

  def __call__(self, dt):    # the forward call
        a = -9.8
        new_v = self.v + a * dt
        new_p = self.p + new_v * dt
        return D(new_p, new_v)

In [139]:
d = D(np.array([0, 0.5]), np.array([1, 1.5]))
for i in range(5):
    d = d(0.01)


In [141]:
d.p, d.v

(array([0.0353, 0.5603]), array([0.51, 1.01]))

# Takeaway message (JIT in a class):

- several methods: helper, self-static, Pytree

- should not use `self.<var>` = in a jitted function (otherwise side effect). By the way, the code analysis by JIT is done via `Traced<ShapedArray>`

- use of `__hash__` and `__eq__`, and the DeviceArray is unhastable

- when the compilation is triggered (JIT)

- See also the doc [to-jit-or-not-to-jit](https://jax.readthedocs.io/en/latest/notebooks/thinking_in_jax.html?highlight=Traced%3CShapedArray%3E#to-jit-or-not-to-jit)

- Ask yourself "Do I need OO encapsulation (i.e. make Class) ?"